In [7]:
import sympy as sp

v_spt, v_lv, v_rv, e_t = sp.symbols('v_spt v_lv v_rv e_t')

In [18]:
from biomechanical_models import base_classes

base_classes.torch = sp

class PressureVolume(base_classes.PressureVolume):
    def __init__(self, name: str):
        self.e_es, self.v_d, self.v_0, self.lam, self.p_0 = sp.symbols(f'{name}.e_es {name}.v_d {name}.v_0 {name}.lam {name}.p_0')

    def p_ed(self, v):
        return self.p_0 * self.lam * (v - self.v_0)

spt = PressureVolume('spt')
lvf = PressureVolume('lvf')
rvf = PressureVolume('rvf')

In [19]:
v_lvf = v_lv - v_spt
v_rvf = v_rv + v_spt

p_lvf = e_t * lvf.p_es(v_lvf) + (1 - e_t) * lvf.p_ed(v_lvf)
p_rvf = e_t * rvf.p_es(v_rvf) + (1 - e_t) * rvf.p_ed(v_rvf)
p_spt = e_t * spt.p_es(v_spt) + (1 - e_t) * spt.p_ed(v_spt)

p_spt_rhs = p_lvf - p_rvf


In [21]:
eq = sp.Eq(p_spt, p_spt_rhs)
eq.simplify()

Eq(e_t*spt.e_es*(spt.v_d - v_spt) - spt.lam*spt.p_0*(e_t - 1)*(spt.v_0 - v_spt), e_t*lvf.e_es*(lvf.v_d - v_lv + v_spt) + e_t*rvf.e_es*(-rvf.v_d + v_rv + v_spt) - lvf.lam*lvf.p_0*(e_t - 1)*(lvf.v_0 - v_lv + v_spt) - rvf.lam*rvf.p_0*(e_t - 1)*(-rvf.v_0 + v_rv + v_spt))

In [29]:
sol = sp.solve(eq, v_spt)[0]
sol

(-e_t*lvf.e_es*lvf.v_d + e_t*lvf.e_es*v_lv + e_t*lvf.lam*lvf.p_0*lvf.v_0 - e_t*lvf.lam*lvf.p_0*v_lv + e_t*rvf.e_es*rvf.v_d - e_t*rvf.e_es*v_rv - e_t*rvf.lam*rvf.p_0*rvf.v_0 + e_t*rvf.lam*rvf.p_0*v_rv + e_t*spt.e_es*spt.v_d - e_t*spt.lam*spt.p_0*spt.v_0 - lvf.lam*lvf.p_0*lvf.v_0 + lvf.lam*lvf.p_0*v_lv + rvf.lam*rvf.p_0*rvf.v_0 - rvf.lam*rvf.p_0*v_rv + spt.lam*spt.p_0*spt.v_0)/(e_t*lvf.e_es - e_t*lvf.lam*lvf.p_0 + e_t*rvf.e_es - e_t*rvf.lam*rvf.p_0 + e_t*spt.e_es - e_t*spt.lam*spt.p_0 + lvf.lam*lvf.p_0 + rvf.lam*rvf.p_0 + spt.lam*spt.p_0)

In [81]:
sol.factor(lvf.e_es, lvf.p_0, lvf.lam, lvf.v_0, rvf.e_es, rvf.p_0, rvf.lam, rvf.v_0, spt.e_es, spt.p_0, spt.lam, spt.v_0)

-(-e_t*spt.e_es*spt.v_d + lvf.e_es*(e_t*lvf.v_d - e_t*v_lv) + lvf.lam*lvf.p_0*lvf.v_0*(1 - e_t) + lvf.lam*lvf.p_0*(e_t*v_lv - v_lv) + rvf.e_es*(-e_t*rvf.v_d + e_t*v_rv) + rvf.lam*rvf.p_0*rvf.v_0*(e_t - 1) + rvf.lam*rvf.p_0*(-e_t*v_rv + v_rv) + spt.lam*spt.p_0*spt.v_0*(e_t - 1))/(e_t*lvf.e_es + e_t*rvf.e_es + e_t*spt.e_es + lvf.lam*lvf.p_0*(1 - e_t) + rvf.lam*rvf.p_0*(1 - e_t) + spt.lam*spt.p_0*(1 - e_t))